In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableBranch

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(
    model="qwen3-max-2026-01-23",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0,
)


## --- 定义模拟子 Agent 处理程序（相当于 ADK 的 sub_agents） ---
def booking_handler(request: str) -> str:
    """模拟预定 Agent 处理请求。"""
    print("\n--- 委托给预定处理程序 ---")
    return f"预定处理程序处理了请求：'{request}'。结果：模拟预定操作。"


def info_handler(request: str) -> str:
    """模拟信息 Agent 处理请求。"""
    print("\n--- 委托给信息处理程序 ---")
    return f"信息处理程序处理了请求：'{request}'。结果：模拟信息检索。"


def unclear_handler(request: str) -> str:
    """处理无法委托的请求。"""
    print("\n--- 处理不清楚的请求 ---")
    return f"协调器无法委托请求：'{request}'。请澄清。"


## --- 定义协调器路由链（相当于 ADK 协调器的指令） ---
## 此链决定应委托给哪个处理程序。
coordinator_router_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """分析用户的请求并确定哪个专家处理程序应处理它。
- 如果请求与预定航班或酒店相关，输出 'booker'。
- 对于所有其他一般信息问题，输出 'info'。
- 如果请求不清楚或不适合任一类别，输出 'unclear'。
只输出一个词：'booker'、'info' 或 'unclear'。""",
        ),
        ("user", "{request}"),
    ]
)

coordinator_router_chain = coordinator_router_prompt | llm | StrOutputParser()

## --- 定义委托逻辑（相当于 ADK 的基于 sub_agents 的自动流） ---
## 使用 RunnableBranch 根据路由链的输出进行路由。
## 为 RunnableBranch 定义分支
branches = {
    "booker": RunnablePassthrough.assign(
        output=lambda x: booking_handler(x["request"]["request"])
    ),
    "info": RunnablePassthrough.assign(
        output=lambda x: info_handler(x["request"]["request"])
    ),
    "unclear": RunnablePassthrough.assign(
        output=lambda x: unclear_handler(x["request"]["request"])
    ),
}

## 创建 RunnableBranch。它接受路由链的输出
## 并将原始输入（'request'）路由到相应的处理程序。
delegation_branch = RunnableBranch(
    (
        lambda x: x["decision"].strip() == "booker",
        branches["booker"],
    ),  # 添加了 .strip()
    (lambda x: x["decision"].strip() == "info", branches["info"]),  # 添加了 .strip()
    branches["unclear"],  # 'unclear' 或任何其他输出的默认分支
)

## 将路由链和委托分支组合成单个可运行对象
## 路由链的输出（'decision'）与原始输入（'request'）一起传递
## 到 delegation_branch。
coordinator_agent = (
    {
        "decision": coordinator_router_chain,
        "request": RunnablePassthrough(),
    }
    | delegation_branch
    | (lambda x: x["output"])
)  # 提取最终输出


## --- 示例用法 ---
def main():
    print("--- 运行预定请求 ---")
    request_a = "给我预定去伦敦的航班。"
    result_a = coordinator_agent.invoke({"request": request_a})
    print(f"最终结果 A：{result_a}")

    print("\n--- 运行信息请求 ---")
    request_b = "意大利的首都是什么？"
    result_b = coordinator_agent.invoke({"request": request_b})
    print(f"最终结果 B：{result_b}")

    print("\n--- 运行不清楚的请求 ---")
    request_c = "告诉我关于abc的事。"
    result_c = coordinator_agent.invoke({"request": request_c})
    print(f"最终结果 C：{result_c}")


if __name__ == "__main__":
    main()

--- 运行预定请求 ---

--- 委托给预定处理程序 ---
最终结果 A：预定处理程序处理了请求：'给我预定去伦敦的航班。'。结果：模拟预定操作。

--- 运行信息请求 ---

--- 委托给信息处理程序 ---
最终结果 B：信息处理程序处理了请求：'意大利的首都是什么？'。结果：模拟信息检索。

--- 运行不清楚的请求 ---

--- 处理不清楚的请求 ---
最终结果 C：协调器无法委托请求：'告诉我关于abc的事。'。请澄清。
